### Parameters

### Verificação dos parâmetros

### Rings

In [1]:
import hashlib
import random
import math

In [2]:
K = 256 
k = 1 # sample
n = 512 # dimensão
q = 4205569 # módulo 
h = 30
L_E = 1586 # limite de E
L_S = 1586 # limite de S
B = (2**20)-1 # intervalo de aleatoriedade na assinatura
d = 21 # #bits aleatórios

Zx.<x> = ZZ[]
R.<x> = Zx.quotient(x^n + 1)
print(R)

Fq = GF(q)
Fqy.<y> = Fq[]
Rq.<y> = Fqy.quotient(y^n + 1)
print(Rq)

def verify_parameters():
    try:
        assert (q.is_prime() and is_odd(q))
        print('Parameters are valid!')
    except:
        print('Error in parameter **q**')

verify_parameters()

Univariate Quotient Polynomial Ring in x over Integer Ring with modulus x^512 + 1
Univariate Quotient Polynomial Ring in y over Finite Field of size 4205569 with modulus y^512 + 1
Parameters are valid!


## Algorithms

# KEY GEN

In [3]:
def Gen_A():
    return Rq.random_element()

def checkE(e):
    my_sum = 0
    e_list = list(e)
    e_list.sort(reverse=True)
    for i in range(0,h):
        my_sum += e_list[i]
    
    if my_sum > L_E:
        return 1
    return 0


def checkS(s):
    my_sum = 0
    s_list = list(s)
    s_list.sort(reverse=True)
    for i in range(0,h):
        my_sum += s_list[i]
    
    if my_sum > L_S:
        return 1
    return 0

def generate_S():
    s = None
    while True:
        s = Rq.random_element(distribution="gaussian")
        if checkS(s) == 0:
            return s
    
def generate_E():
    e = None
    while True:
        e = Rq.random_element(distribution="gaussian")
        if checkE(e) == 0:
            return e

In [4]:
def key_generation():
    print('\n--- Starting KeyGen Process ---\n')
    
    '''
    counter = 1
    pre_seed = bytearray(os.urandom(K))
    #print(pre_seed)
    print('We have **pre_seed**')
    
    
    seeds = PRF1(pre_seed) #devolve 7 seeds de 256bits cada
    #print (len(seeds))
    #print(seeds)
    print('We have **seeds**')
    
    seed_s = seeds[0]
    #print(seed_s)
    seeds_e = seeds[1:len(seeds)-2]
    seed_a = seeds[len(seeds)-2]
    seed_y = seeds[len(seeds)-1]
    #print(len(seed_y))
    '''
    a = Gen_A()
    #print(a)
    print('We have **polynomial a**')
    
    (s) = generate_S()
    print('We have **s**')
    
    (e) = generate_E()
    print('We have **e**')
    
    t = a*s + e
    print('We have **t**')
    
    sk = (s,e,a)
    print('We have **sk**')
    pk = (t,a)
    print('We have **pk**')
    
    print('\n--- KeyGen Process Done ---\n')
    
    return sk,pk

(sk,pk) = key_generation()


--- Starting KeyGen Process ---

We have **polynomial a**
We have **s**
We have **e**
We have **t**
We have **sk**
We have **pk**

--- KeyGen Process Done ---



In [83]:
def lift_at_center(x):
    center = q//2
    return lift(x+center) - center

def poly_round_to_center(poly):
    coefs = poly.list()
    ret = []
    for coef in coefs:
        ret.append(Zx(lift_at_center(coef)))
    return ret

def list_round_to_center(coefs):
    ret = []
    for coef in coefs:
        ret.append(Zx(lift_at_center(coef)))
    return ret

from sage.cpython.string import bytes_to_str, str_to_bytes

def hash(m):
    new_m = str_to_bytes(m)
    h = hashlib.sha256()
    h.update(new_m)
    return h.digest()

from sage.crypto.util import ascii_integer
def something_to_bin(inputs):
    bin = BinaryStrings()
    return list(bin.encoding(str(inputs)))


def H(v,hash_m):
    w = [None] * n
    for i in range(n):
        val = v[i] % 2^d
        if(val > 2^(d-1)):
            val = val - 2^d
        w[i] = (v[i] - val)/ (2^d)
    hash_m_list = something_to_bin(hash_m)
    if(len(w)>len(hash_m_list)):
        hash_m_list += [0]*(len(w)-len(hash_m_list))
    elif(len(w)<len(hash_m_list)):
        w +=[0]*(len(hash_m_list)-len(w))
    else:
        print('Tamanho está bom.')
    
    return w + hash_m_list

# SIGNING

In [84]:
def calculate_Z(y,s,c):
    #assert len(s.list()) == len(y.list()) == len(c)
    list_y = y.list()
    list_s = s.list()
    
    z = [None] * n
    for i in range (0,n):
        z[i] = list_y[i] + list_s[i]*c[i]
    return z

In [90]:
def sign_message(m,sk):
    print('\n--- Starting Signing Message Process ---\n')
    (s,e,a) = sk
    
    y = Rq.random_element(x=-B, y = B+1, distribution='uniform')
    print('We have **y**')
    #print(y)
    
    v = poly_round_to_center(a*y)
    print('We have **v**')
    #print(v)
    
    hash_m = hash(m)
    c = H(v,hash_m)
    print(len(c))
    print('We have **c**')
    #print(c1)
    #print(s)
    
    z = calculate_Z(y,s,c)
    print('We have **z**')
    
    print('\n--- Signing Message Process Done ---\n')
    return (z,c,hash_m)

signature = sign_message('test',sk)
#print(signature)


--- Starting Signing Message Process ---

We have **y**
We have **v**
1584
We have **c**
We have **z**

--- Signing Message Process Done ---



# VERIFY

In [91]:
def calculate_W(a,t,z,c):
    poly_z = Rq([z[i] for i in range(n)])
    poly_c = Rq([c[i] for i in range(n)])
    #assert len(a.list()) == len(t.list()) == len(z) == len(c) == n
    
    left = a*poly_z
    #print(left)
    
    right = t*poly_c
    
    #poly = Zx([left_minus_right[i] for i in range(n)])
    #print (poly)
    
    w = poly_round_to_center( left-right)
    return w

In [94]:
def verify_signature(signature, pk):
    print('\n--- Starting Signature Verification Process ---\n')
    
    (z,c,hash_m) = signature
    (t,a) = pk
    #print(t)
    #print(a)
    
    w = calculate_W(a,t,z,c)
    
    hw = H(w,hash_m)
    #print(hw)
    
    print(c)
    print(hw)
    
    if c!=hw:
        print('\n--- Signature Verification Process Done ---\n')
        return False

    else:
        print('\n--- Signature Verification Process Done---\n')
        return True
    
    
    
    
verify_signature(signature, pk)


--- Starting Signature Verification Process ---

[0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1

False

## Operações auxiliares

In [ ]:
'''
def SHAKE(X,L,S = None):
    if S is None:
        return hashlib.shake_256(b'0x01'+X).digest(int(L))

def PRF1(X):
    seeds_length = K* (k +3)
    aux = SHAKE(X,seeds_length)
    seeds = []
    for i in range(0,seeds_length,K):
        seeds.append(aux[i:i+K])
    ret = []
    for seed in seeds:
        aux = []
        for s in seed:
            aux.append(R(s))
        ret.append(aux)
    return ret

def Gen_A2(seed_a):
    assert len(seed_a) == K
    aux = SHAKE(seed_a,k)
    ret = []
    for value in aux:
        ret.append(value)
    return Rq(ret)

def is_odd(number):
    if (mod(number,2)!= 0):
        return True
    else:
        return False
        
def list_abs(e):
    for i in range(0,len(e)):
        e[i] = abs(int(e[i]))
    return e
'''